# Maldives covid curve

**Code taken from 
https://joachim-gassen.github.io/tidycovid19/
Packages used: tidyverse,tidycovid19,zoo**

In [2]:
install.packages("tidyverse")
install.packages("tidycovid19")
install.packages("zoo")

library(tidyverse)
library(tidycovid19)
library(zoo)

## Load John Hopkins COVID 19 data:

In [ ]:
df <- download_merged_data(cached = TRUE, silent = TRUE)

**Filter for Maldives to obtain the number of contagious for this country.
Calculate the number of daily new cases (confirmed) and a rolling average.**

In [ ]:
df %>%
  filter(country == "Maldives") %>%
  mutate(
    new_cases = confirmed - lag(confirmed),
    ave_new_cases = rollmean(new_cases, 3, na.pad=TRUE, align="right")
  ) %>%
  filter(!is.na(new_cases), !is.na(ave_new_cases)) %>%

**Plot the data from new cases and average new cases** 

In [ ]:

  ggplot(aes(x = date)) +
  geom_bar(aes(y = new_cases), stat = "identity", fill = "lightblue") +
  geom_line(aes(y = ave_new_cases), color ="red") +
  scale_x_date(breaks = as.Date(c("2020-03-08", "2020-04-15", "2020-06-08", "2020-07-08","2020-07-23","2020-09-27","2020-12-07")),
               minor_breaks = as.Date(c("2020-04-15", "2020-07-08")))+
  theme_minimal()


# Maldives Coronavirus Disease Prevention Map

Facebook Population (Tile Level)

Description:Location density maps are heat maps, which show where people are located before, during and after a disaster and where populations have increased or decreased. We can compare this information to historical records, like population estimates based on satellite images. Comparing these data sets can help response organizations understand areas impacted by a natural disaster. The following metrics are are included in the map: * Date Time - The time period represented by the current map layer. * Standard (Z) Score - The number of standard deviations by which the crisis population count in the location differs from the baseline count. * Baseline:People - The average number of people we expect to be in the area during the specified time based on pre-disaster estimates. * Crisis:People - The number of people observed in the tile during the selected time period. * Difference - The difference between the population at the time of the crisis and the population during the baseline. * Percent Change - The percentage difference between the population at the time of the crisis and the population during the baseline. For more information go to the Help Page https://fburl.com/disastermaps_help

The Facebook population maps were downloaded from:
https://www.facebook.com/geoinsights-portal/
and then uploaded to the DPA drive folder: 
https://drive.google.com/drive/u/0/folders/117og1_1L1ht0vUEhOXj19yJXlGv4sgcI

**Read and bind all the csv cointained in the corresponding folder**

In [ ]:
options(scipen=999) 
batch_read <- function(path, pattern, recursive = FALSE, read_fun, ...) {
  data.files <- list.files(path, pattern = pattern, recursive = recursive)
  data <- lapply(paste0(path, data.files), read_fun, ...)
  data <- do.call("rbind", data)
  data
}

idb <- batch_read(
  path = "./populationFB/data/"
  , pattern = "\\.csv"
  , read_fun = read.csv
  , header = TRUE
)

**For Kepler to display a temporal visualization of data
A date_time columns needs to be included in the following format:
yyyy-mm-dd 00:00:00.0
the following script adds a synthetic column named millisec 
that is pasted to the date column**

In [ ]:
idb$date <- as.factor(sapply(as.character(idb$date_time), function(x) {strsplit(x, "\\ ")[[1]][1]}))
df<-idb[,c(1,2,3,5,11,14)]
df$millisec <- cumsum(c(0,as.numeric(diff(df$date))!=0))
df$time<-rep("00:00:01.", nrow(df))
df$millisec<-paste(df$time, df$millisec, sep="")
df$time<-NULL

df$date<-as.character(df$date)
df$dateTime<-paste(df$date, df$millisec, sep=" ")
df<-df[,c(1,2,3,5,8)]

write.csv(df, "Preprocessed_data_for_kepler.csv", row.names=FALSE)

**Then use this csv in https://kepler.gl/**

**See https://docs.google.com/document/d/1qyuaCTgDr5b6ACVVbXanYrA-PNHVsuKu5JeH4zOUGao/edit**

# Maldives Mobility percent change

Data from Movement Between Tiles Facebook Data

Description:What specific pairs of map tiles are people moving between more or less often than we would expect based on pre-crisis levels? This dataset contains information about the number of people moving between tile pairs over a given time period. We measure this during baseline (movement between tile pairs averaged across the three weeks prior to the disaster) as well, so we can understand how many more or fewer people are moving during the disaster period compared to usual. This helps us distinguish disaster related movements from people’s normal migration patterns. The following metrics are available: * Date Time - The time period represented by the current map layer. * Starting Location - The region or tile where the movement of the group started. * Ending Location: The region or tile where the movement of the group ended. * Length (km) - The distance traveled in kilometers. * Baseline: People Moving - The total number of people who moved from Starting Location to Ending Location on average during the weeks before the disaster began. * Crisis: People Moving - The total number of people who moved from Starting Location to Ending Location during the time period specified * Difference - The difference between the number of people moving from Starting Location to Ending Location during the disaster compared to before the disaster. * Percent Change - The percentage difference between the number of people moving from Starting Location to Ending Location during the disaster compared to before the disaster. * Standard (Z) Score: The number of standard deviations by which the count of people moving during the crisis differs from the number of people moving during the baseline. Any z-value greater than 4 or smaller than -4 is clipped at 4 or -4. For more information go to the Help Page https://fburl.com/disastermaps_help

In [ ]:
options(scipen=999)  # turn-off scientific notation like 1e+48

**Read all csv**

In [ ]:
batch_read <- function(path, pattern, recursive = FALSE, read_fun, ...) {
  data.files <- list.files(path, pattern = pattern, recursive = recursive)
  data <- lapply(paste0(path, data.files), read_fun, ...)
  data <- do.call("rbind", data)
  data
}

#
rdb <- batch_read(
  path = "./BetwTiles/"
  , pattern = "\\.csv"
  , read_fun = read.csv
  , header = TRUE
)

**Process and clean the data**

In [ ]:
db<-rdb[,c(1,2,4,6,7,11:14,16:20)]


db$origen <- as.factor(sapply(as.character(db$geometry), function(x) {strsplit(x, "\\,")[[1]][1]}))
db$destino <- as.factor(sapply(as.character(db$geometry), function(x) {strsplit(x, "\\,")[[1]][2]}))
db$geometry<-NULL

db$origen <- as.character(db$origen)
db$destino <- as.character(db$destino)

geom<-db[,c(14,15)]

#write.csv(geom, "./BetwTiles_rdb.csv", row.names=FALSE)

**Read the clean data**

In [ ]:
geom<-read.csv("BetwTiles_rdb2_curated.csv")

**Remove dispensable coordinates**

In [ ]:
db<-db[,c(1:9)]

**Join with recent extracted and curated coordinates**

In [ ]:
dbb<-cbind(db,geom)

**Add identifier**

In [ ]:
dbb$ide<-seq.int(nrow(dbb))
#write.csv(dbb, "./BetwTiles_rdb_geometries.csv", row.names=FALSE)

**Reading the clean data**

In [ ]:
dbb<-read.csv("BetwTiles_rdb_geometries.csv")

**Assign coordinates into Atolls
 using shapefiles downloaded from 
 https://data.humdata.org/dataset/maldives-administrative-boundaries-polygon-polyline
 Download the one named "mdv_devinfo_admin2b"**


In [ ]:
#install.libraries("rgdal")
library(rgdal)

setwd("./shapefiles/")
geo <- readOGR(dsn="mdv_devinfo_admin2b", layer="MDV_DevInfo_Admin2B")
geo <- geo[,c(1,2,3,9)]
plot(geo)

library(ggplot2)
geodf<-fortify(geo, region="ID_")
head(geodf)
geo$id <- row.names(geo)  

**Allocate an id variable to the sp data
Spatial Allocatio in 
Use first lng**

In [ ]:
coords<-dbb[,c(10,11)]

**Set as spatial object**

In [ ]:
sp <- SpatialPoints(coords)
rm(coords)

**Use the following Coordinate Reference System (CRS)**

In [ ]:
|proj4string(sp) <- "+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0"
proj4string(geo) <-"+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0"

**Assigning points into polygons**

In [ ]:
assign <- over(sp, geo) 
dim(assign)

**Use rownames just after over function to use the in the following merge**

In [ ]:
assign$rous <- rownames(assign)
dbb$rous<-rownames(dbb)

assign$rous<-as.factor(as.character(assign$rous))
dbb$rous<-as.factor(as.character(dbb$rous))
names(assign)[1]<-"ID"

fdf <- merge(x=assign, y=dbb, by.x="rous", by.y="rous")

#fdf<- fdf[!is.na(fdf$ID),]
#NAs can be removed, prefer not, if necessary these will be named as "Indian Ocean", 
#which is the original name in the raw data

**Rename these columns**

In [ ]:
names(fdf)[3]<-"origen1"
names(fdf)[4]<-"origen3"
names(fdf)[5]<-"origen0"

**Second Assign coordinates into Atolls for the destination coordinates**


In [ ]:
setwd("./shapefiles/")
geo <- readOGR(dsn="mdv_devinfo_admin2b", layer="MDV_DevInfo_Admin2B")
geo <- geo[,c(1,2,3,9)]
plot(geo)

library(ggplot2)
geodf<-fortify(geo, region="ID_")
head(geodf)
geo$id <- row.names(geo) # allocate an id variable to the sp data

**Second Spatial Allocatio in First lng**

In [ ]:
coords<-dbb[,c(12,13)]

**Set as spatial object**

In [ ]:
sp <- SpatialPoints(coords)
rm(coords)

**CRS**

In [ ]:
proj4string(sp) <- "+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0"
proj4string(geo) <-"+proj=longlat +datum=WGS84 +no_defs +ellps=WGS84 +towgs84=0,0,0"

assign <- over(sp, geo) 
dim(assign)

**Get rownames just after over function**

In [ ]:
assign$rous <- rownames(assign)
dbb<-dbb[,c(3,12:14)]
dbb$rous<-rownames(dbb)

assign$rous<-as.factor(as.character(assign$rous))
dbb$rous<-as.factor(as.character(dbb$rous))
names(assign)[1]<-"ID"

fdf2 <- merge(x=assign, y=dbb, by.x="rous", by.y="rous")

#fdf2<- fdf2[!is.na(fdf2$ID),]
#NAs can be removed, prefer not, if necessary these will be named as "Indian Ocean", 
#which is the original name in the raw data

names(fdf2)[3]<-"destination1"
names(fdf2)[4]<-"destination3"
names(fdf2)[5]<-"destination0"


fin <- merge(x=fdf, y=fdf2, by.x="ide", by.y="ide")

final<-fin[,c(1,4,5,6,8:20,23,24,25)]

#write.csv(final, "/home/memo/Documents/perso/DataPopAlliance/maldives/dir/geoinsights/BetwTiles_rdb_official_polygons.csv", row.names=FALSE)

**Read final data**

In [ ]:
final<-read.csv("./BetwTiles_rdb_official_polygons.csv")

**From final object, subset only intra Atolls movement
not between diff Atolls
internalMobility(db0)**

In [ ]:
im<-subset(final, origen1==destination1)

library(lubridate)
im$date <- as.factor(sapply(as.character(im$date_time), function(x) {strsplit(x, "\\ ")[[1]][1]}))
im$time <- as.factor(sapply(as.character(im$date_time), function(x) {strsplit(x, "\\ ")[[1]][2]}))


median<-aggregate(percent_change~date, FUN=median, data=im, na.rm=TRUE)
median$date<-as.Date(median$date, format="%Y-%m-%d")

library(ggplot2)
png("./interMobility_perc_change_median.png", width = 29, height = 8, units = 'in', res = 300)
ggplot(median, aes(x=date, y=percent_change, color=percent_change)) +
  geom_point(size=4) +
  theme(text = element_text(size=30))+
  xlab("Date") + ylab("Mobility Percent Change")+
  geom_smooth(span = 0.3)+
  scale_x_date(breaks = as.Date(c("2020-04-15", "2020-06-08","2020-07-23","2020-09-27","2020-12-07")),
               minor_breaks = as.Date(c("2020-04-15", "2020-07-23")))
  #theme_minimal()
dev.off()

**Since we are interested in plotting for all the atolls, we use origen1, but it could also be destination1 column.**

In [1]:
atollsMedian<-aggregate(percent_change~date+origen1, FUN=median, data=im, na.rm=TRUE)
atollsMedian$date<-as.Date(atollsMedian$date, format="%Y-%m-%d")

library(dplyr)

png("./interMobility_perc_change_median_perAtoll.png", width = 9, height = 6, units = 'in', res = 300)
atollsMedian %>%
  ggplot(aes(x = date, y = percent_change)) + 
  geom_point(alpha = 0.25) +
  facet_wrap(~ origen1, ncol = 4)+
  xlab("Date") +
  ylab("Mobility Percent Change")+
  theme_minimal()+
  geom_smooth(span = 0.3)
dev.off()

ERROR: Error in library(rgdal): there is no package called ‘rgdal’


#  Median change in mobility over 2020 at the Atoll level.

**Continue from previous object. (see Maldvs_MovBetTiles_Figs_6_and_7.R) or read it, if written**

In [ ]:
final<-read.csv("./BetwTiles_rdb_official_polygons.csv")

Internal Mobility (im), mobility intra Atolls, not between diff Atolls

In [ ]:
im<-subset(final, origen1==destination1)


im$date <- as.factor(sapply(as.character(im$date_time), function(x) {strsplit(x, "\\ ")[[1]][1]}))
im$time <- as.factor(sapply(as.character(im$date_time), function(x) {strsplit(x, "\\ ")[[1]][2]}))


**Both origen1 and destination1 are the same**

In [ ]:
imMedian<-aggregate(percent_change~origen1, FUN=median, data=im, na.rm=TRUE)

**Cloropleth**

**Download shapefile from 
https://drive.google.com/drive/u/0/folders/1FUpQE3G4V193xM0_z3u_6OFOSEiyCx7y**

In [ ]:
setwd("./shapefiles/")
geo <- readOGR(dsn="mdv_devinfo_admin2b", layer="MDV_DevInfo_Admin2B")
geo<-geo[,c(1,2)]

**Load libraries**

In [ ]:
library("rgdal")
library(data.table)
library(plyr)
library(ggthemes)
library(data.table)
library(ggplot2)
library(viridis)

In [ ]:
shapefile<-geo
#shapefile = readOGR(dsn = "DIRECTORY WITH SHAPEFILES", layer = "THE ACTUAL SHAPEFILE")
shapefile@data$id = rownames(shapefile@data)
shapefile.points = fortify(shapefile, region = "id")
shapefile.df = join(shapefile.points, shapefile@data, by = "id")
shapefile.df = subset(shapefile.df, select = c(long, lat, group, NAME1_))
names(shapefile.df) = c("long", "lat", "group", "NAME1")
names(imMedian)[1]<-"NAME1"
full.data = join(imMedian, shapefile.df, by = "NAME1", type = "full")

svg("./choropleth_mobility_02c.svg", width = 8, height = 8)
ggplot(full.data, aes(x = long, y = lat, group = group,
       fill = percent_change)) +
    geom_polygon(color = "grey75", size = 0.2) +
    coord_equal() +
    #scale_fill_viridis() +
    #geom_path(colour="black", lwd=0.05) +
    #facet_wrap(~ variable ) +
    #theme_map() + facet_wrap(~ variable)+
    #scale_fill_distiller() +
    #labs(title = "2 halft(CARPETAS DE INVESTIGACIÓN 2020)",
    #     fill = NULL) +
    #theme_void() +
    theme(legend.position = "bottom",
          panel.background = element_rect(fill = NA, colour = "#cccccc"))+ 
    scale_fill_viridis(option = "cividis", direction = -1)
dev.off()

# Encounter probabilities among the Atolls

**Colocation data from geoinsights facebook**

Turn-off scientific notation like 1e+48

In [ ]:
options(scipen=999)  

**Read all csv**

In [ ]:
batch_read <- function(path, pattern, recursive = FALSE, read_fun, ...) {
  data.files <- list.files(path, pattern = pattern, recursive = recursive)
  data <- lapply(paste0(path, data.files), read_fun, ...)
  data <- do.call("rbind", data)
  data
}

rdb <- batch_read(
  path = "./colocation/"
  , pattern = "\\.csv"
  , read_fun = read.csv
  , header = TRUE
)

db<-rdb[,c(2,8,13,15)]

library(lubridate)

**Prepare columns in date format and weeks**

In [ ]:
db$date<-as.Date(db$ds, format="%Y-%m-%d")
db$Week_Day<- as.numeric(format(db$date, format='%w'))
db$week <- db$date + (0 - db$Week_Day)

**Get the mean of the link_value (probability of encounter)**

In [ ]:
dbb<-aggregate(link_value~polygon1_name+polygon2_name+week, FUN=mean, data=db, na.rm=TRUE)

**Weeks of interest from covid curve, same weeks used before**

In [ ]:
w1<-subset(dbb, week=="2020-03-29")
w2<-subset(dbb, week=="2020-05-10")
w3<-subset(dbb, week=="2020-06-07")
w4<-subset(dbb, week=="2020-08-16")
ws<-rbind(w1,w2,w3,w4)
ws<-ws[order(-ws$link_value),]
ws$id<-seq.int(nrow(ws))

**Visually identified the rows where the origen and destination atoll are the same
they finish in row 61**

In [ ]:
mi<-ws[1:61,]

**Prepare data to combine it according to the week of interest**

In [ ]:
m1<-subset(mi, week=="2020-03-29")
m2<-subset(mi, week=="2020-05-10")
m3<-subset(mi, week=="2020-06-07")
m4<-subset(mi, week=="2020-08-16")

m1$Encounter<-paste(m1$polygon1_name, m1$polygon2_name, sep="-")
m2$Encounter<-paste(m2$polygon1_name, m2$polygon2_name, sep="-")
m3$Encounter<-paste(m3$polygon1_name, m3$polygon2_name, sep="-")
m4$Encounter<-paste(m4$polygon1_name, m4$polygon2_name, sep="-")

f1<-m1[,c(6,3,4)]
f2<-m2[,c(6,3,4)]
f3<-m3[,c(6,3,4)]
f4<-m4[,c(6,3,4)]


names(f1)[3]<-"March_29"
names(f2)[3]<-"May_10"
names(f3)[3]<-"June_07"
names(f4)[3]<-"August_16"

f1$week<-NULL
f2$week<-NULL
f3$week<-NULL
f4$week<-NULL

**Merge weeks data, get the mean and print a table**

In [ ]:
final <- merge(x=f1, y=f2, by.x="Encounter", by.y="Encounter")
final <- merge(x=final, y=f3, by.x="Encounter", by.y="Encounter")
final <- merge(x=final, y=f4, by.x="Encounter", by.y="Encounter")

final<-final[order(-final$March_29),]
final[,-1] <-round(final[,-1],3) 
final$mean <- rowMeans(subset(final, select=c(2:5)), na.rm = TRUE)


library(gridExtra)
png("final_coloction_same_atoll.png", height=10, width=10, units = 'in', res = 300)
grid.table(final)
dev.off()